<a href="https://colab.research.google.com/github/nightted/Travel-Recommend-project/blob/main/Booking_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO LIST

1. #TODO : hotel 的 parameters　不一定每一間都會有 (EX: 星等 , 訂房熱度..) , 需要一間一間處理 HTML (Done)
2. #TODO : parameters 的 HTML　有的還沒處理並抓出精華 (Done)
3. #TODO : room information extract
4. #TODO : 飯店附近推薦美食 , 需用 googlesearch package 來抓google搜尋blog結果  




In [3]:
import numpy as np
import pickle 
import requests
from bs4 import BeautifulSoup
import time 
import os
import matplotlib.pyplot as plt

In [4]:
def save_pkl(path  , data):
    with open(path , "wb") as pkl :
        pickle.dump(data , pkl)
        
def load_pkl(path):
    with open(path , "rb") as pkl :
        data = pickle.load(pkl) 
    return data

def StrToDigit(Number):
 
  Number = ''.join(Number.split(','))
  return int(Number)

In [5]:
hotel_data = load_pkl('/content/drive/My Drive/Colab Notebooks/Travel Recommend Project/City data/Tainan/data_hotel')
name_transform = load_pkl('/content/drive/My Drive/Colab Notebooks/Travel Recommend Project/City data/Tainan/Hotel_name_transform.pickel')

# Get Hotels pages data

In [ ]:
# HTML structure 
# <div class="sr_item_content sr_item_content_slider_wrapper "> 下包含 hotel inform main block 
# <div class="sr_item_photo sr_card_photo_wrapper" id="hotel_5621655"> 下包含 hotel pictures

In [55]:
url = "https://www.booking.com/searchresults.zh-tw.html"

#INPUT type scrape_time : ['2020-12-09' ,'2020-12-10']
def get_header_payload(scrape_time , target_hotel = None ):

  '''
  # function : get the headers and payload by scrape time

  target_hotel : if it's not assigned , scarpe all "台南" hotel 

  '''

  check_in_time = scrape_time[0].split('-')
  check_out_time = scrape_time[1].split('-')
  
  payload = {
    'checkin_year': check_in_time[0] ,
    'checkin_month': check_in_time[1] ,
    'checkin_monthday': check_in_time[2] ,
    'checkout_year': check_out_time[0] ,
    'checkout_month': check_out_time[1] ,
    'checkout_monthday': check_out_time[2] ,
    'aid': '1288258', # need or not?
    'label': 'label: metagha-link-mapresultsTW-hotel-2468603_dev-desktop_los-1_bw-18_dow-Monday_defdate-0_room-0_lang-zh_curr-TWD_gstadt-2_rateid-0_aud-0_cid-_gacid-6642513825_mcid-10_ppa-0_clrid-0_ad-1_gstkid-0', # need or not?
    'sid': 'ab79516c4eea1e96b378b9c2338022ce', # need or not?
    'sb': '1', # need or not?
    'src': 'searchresults', 
    'src_elem': 'sb', # need or not?
    'ss': target_hotel if target_hotel else '台南', 
    'ssne': None if target_hotel else '台南', # place you want to search (encode)
    'ssne_untouched': None if target_hotel else '台南' , # place you want to search (encode)
    'is_ski_area': '0', # need or not?
    'dest_type': None if target_hotel else 'city', 
    'dest_id': None if target_hotel else '-2637868', # place id you want to search 
    'group_adults': '2', # number of people
    'group_children': '0', # number of children
    'no_rooms': '1', # number of rooms
    'from_sf': '1', # ???
    'ac_langcode': 'xt'

  }

  headers = {    
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36', # can be generate fake agent ??
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'error_url': "https://www.booking.com/index.zh-tw.html?aid=1288258;label=metagha-link-mapresultsTW-hotel-2468603_dev-desktop_los-1_bw-18_dow-Monday_defdate-0_room-0_lang-zh_curr-TWD_gstadt-2_rateid-0_aud-0_cid-_gacid-6642513825_mcid-10_ppa-0_clrid-0_ad-1_gstkid-0;sid=ab79516c4eea1e96b378b9c2338022ce;sb_price_type=total;srpvid=f1b826d7eb680028&;",
    'content-security-policy-report-only': "report-uri https://csp-receiver.booking.com/csp_violation?type=report&tag=112&pid=040f2610ef80010a&e=UmFuZG9tSVYkc2RlIyh9YRdubXl3m7MIPItwv4TRRzARPrDvNuzlrbfIvmyYAyhkeKKOaHLiHXalz9oYwBjSNw&f=2&s=0; frame-ancestors 'none';",
    'Referer' : 'https://www.booking.com/index.zh-tw.html?aid=376396&label=booking-name-yefrPbbyS%2AFIINHgyCnmNgS267725091255%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1012818%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YfqnDqqG8nt1O4nYvDr1lms&sid=99dd5960476d5bf057cf94fe33e3deb6&srpvid=6d1e3723a19e0104&click_from_logo=1',
    'Host': 'www.booking.com',    
    'Upgrade-Insecure-Requests': '1',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'max-age=0',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1' ,
    'Connection': 'keep-alive'
  }
  
  return payload , headers

# check target information in search result or not
def check_alive_or_not( search_result , text = True , tag  = None , msg_if_none = None ):
  '''
  text : if True , get text in tag ; if False , get tag property

  tag : if not none , get property of that tag
  '''
  if not search_result:
    return msg_if_none
  return search_result.text if text else search_result[tag]


# get information of hotels
def get_hotel_information( hotel_name , date = None ,  instant_information = False ):

  '''
  #function : get hotel information including non-instant or instant data by hotel_name

  hotel_name : enter name of hotel in "booking or agoda" website

  instant_information : if True , get instant information ( price , avaliable number of rooms , room_recommend )

  date : day to check-in and check-out , ex: ['2020-12-10','2020-12-11']

  '''

  date = date if instant_information else ['2020-12-10','2020-12-11']

  payload , headers = get_header_payload( date , target_hotel = hotel_name )
  res = requests.post( url, data = payload , headers = headers)
  res_text = res.content
  soup = BeautifulSoup( res_text , "html.parser")

  soup_content = soup.find_all( "div" , { 'class' : "sr_item_content sr_item_content_slider_wrapper "})[0]
  soup_pic = soup.find_all( "div" , { 'class' : "sr_item_photo sr_card_photo_wrapper"})[0]
    
  return extract_informations_from_soup( soup_content , soup_pic , instant_information )

# extract information from soup
def extract_informations_from_soup( soup_content , soup_pic , instant_information = False ):

  '''
  # function : extract information from soup

  soup_content , soup_pic : content of soup

  instant_information : if True , get instant information ( price , avaliable number of rooms , room_recommend )
  '''

  # p class="simple_av_calendar_no_av sold_out_msg"
  if instant_information :
    ### get below <div class="sr_item_content sr_item_content_slider_wrapper ">

    room_soldout = bool(soup_content.find('p' , { 'class' : "simple_av_calendar_no_av sold_out_msg"}))
    if room_soldout:
      return {'room_recommend' : None ,
           'room_remainings' : None ,
           'hot' : None ,
           'price' : None ,
           'instant_hrefs' : None }

    hotel_room_recommend = soup_content.find('div', {'class':"room_link"}).find('strong').text # get the recommend romm type  (instant information)
    hotel_room_remainings = check_alive_or_not(soup_content.find('span', {'class':"only_x_left sr_rooms_left_wrap "}), msg_if_none = '房源還很充足!' ).rstrip("\n").strip("\n") # get remaining rooms (instant information)
    hotel_hot = check_alive_or_not(soup_content.find('div' , {'class' : 'rollover-s1 lastbooking'}) , msg_if_none = '快上網站訂房!').rstrip("\n").strip("\n") # get the hot of hotels (instant information) 不是每個hotel都有此block !!!!!
    hotel_price = soup_content.find('div' , {'class' : "bui-price-display__value prco-inline-block-maker-helper" }).text.rstrip("\n").strip("\n") # get hotels price (instant information)
    hotel_instant_hrefs = soup_content.find('a', {'class':"js-sr-hotel-link hotel_name_link url"})['href'].strip("\n") # get room_hrefs

    # !!BUGS: price 部分有 /xa0124 encode 的問題
    return {'room_recommend' : hotel_room_recommend ,
          'room_remainings' : hotel_room_remainings ,
          'hot' : hotel_hot ,
          'price' : "".join(hotel_price.split()) , # remove /xa0 
          'instant_hrefs' : hotel_instant_hrefs }


  else :
    ### get below <div class="sr_item_content sr_item_content_slider_wrapper ">
    hotel_name = soup_content.find('span', {'class':"sr-hotel__name "}).string.rstrip("\n").strip("\n") # get hotels
    hotel_hrefs = soup_content.find('a', {'class':"js-sr-hotel-link hotel_name_link url"})['href'].strip("\n") # get room_hrefs 
    # these items below may not exist
    hotel_rating = check_alive_or_not( soup_content.find( "div" , { 'class' : "bui-review-score__badge"}) )
    hotel_comment_num = check_alive_or_not( soup_content.find('div' , {"class" : "bui-review-score__text"}) ) # get # of comments 
    hotel_star = check_alive_or_not( soup_content.find('span' , {'class' : "bui-rating bui-rating--smaller" }) , text = False , tag = 'aria-label') # get star of hotel 

    ### get below <div class="sr_item_photo sr_card_photo_wrapper" id="hotel_5621655">
    hotel_pic_link = soup_pic.find('img' , {'class' : "hotel_image" })['data-highres'] # get hotel review pic

    return {'source_name' : hotel_name ,
        "href": hotel_hrefs , 
        "comment_num": hotel_comment_num ,
        "stars": hotel_star ,
        "pic_link": hotel_pic_link ,
        "source_rating": hotel_rating }


def get_detail_hotel_information( hotel_href ):

  '''
  # function : get detail hotel information by inner pages

  hotel_href : the url to inner pages
  
  '''

  header_url = "https://www.booking.com"
  new_path = header_url + hotel_href

  res = requests.get(new_path)
  res = res.content
  soup = BeautifulSoup(res)

  soup_sub = soup.find('div' , { 'class' : " clearfix bh-photo-grid fix-score-hover-opacity "})
  soup_comment = soup_sub.find_all( 'p' , { 'class' : 'trackit althotelsReview2 fixed_review_height fixed_review_top_align review_content'})
  comments = [ item.text.rstrip("\n").strip("\n") for item in soup_comment ] 

  soup_pic = soup_sub.find_all("a" , { 'class' : 'bh-photo-grid-item'})
  pics = [ item['href'] for item in soup_pic ]

  return { 'comments' : comments , 
        'pics' : pics }
          

In [54]:
# test
for date in range(10,30):
  print(f'date : 2020-12-{date}:',get_hotel_information( hotel_data[193]['source_name'] , instant_information=True, date=[f'2020-12-{date}',f'2020-12-{date+1}']) )

date : 2020-12-10: {'room_recommend': '小型家庭套房', 'room_remainings': '此類選項本站僅剩 6 個', 'hot': '快上網站訂房!', 'price': 'TWD6,160', 'instant_hrefs': '/hotel/tw/tai-nan-jing-ying-jiu-dian.zh-tw.html?aid=1288258&label=label%3A%20metagha-link-mapresultsTW-hotel-2468603_dev-desktop_los-1_bw-18_dow-Monday_defdate-0_room-0_lang-zh_curr-TWD_gstadt-2_rateid-0_aud-0_cid-_gacid-6642513825_mcid-10_ppa-0_clrid-0_ad-1_gstkid-0&sid=d048dc7038c60a1d5256ef245ef13dc5&all_sr_blocks=105532942_135726485_0_2_0&checkin=2020-12-10&checkout=2020-12-11&dest_id=-2637868&dest_type=city&group_adults=2&group_children=0&hapos=1&highlighted_blocks=105532942_135726485_0_2_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=105532942_135726485_0_2_0__616000&srepoch=1605265479&srpvid=faf34de39db9011a&ucfs=1&from=searchresults\n;highlight_room=#hotelTmpl'}
date : 2020-12-11: {'room_recommend': '行政特大雙人床房', 'room_remainings': '房源還很充足!', 'hot': '快上網站訂房!', 'price': 'TWD7,480', 'instant_hrefs': '/hotel/tw/tai-nan-jing-ying-jiu-dian.

# Get hotel main page data 

In [ ]:
# grab hotel main page data 
restart_number = 0
for idx , data_dict in enumerate(hotel_data):
  
  if 'source_name' in data_dict and idx >= restart_number :
    hotel_inform = get_hotel_information(data_dict['source_name']) # return dict
    data_dict.update(hotel_inform)
    print(f' Now in [{idx+1}/{len(hotel_data)}] , finishing rate {(idx+1)/(len(hotel_data))*100} %')
    time.sleep(5)


'\nrestart_number = 0\nfor idx , data_dict in enumerate(hotel_data):\n  \n  if \'source_name\' in data_dict and idx >= restart_number :\n    hotel_inform = get_hotel_information(data_dict[\'source_name\'])\n    data_dict.update({\n      "href" : hotel_inform[1] ,\n      "comment_num" : hotel_inform[2] ,\n      "stars" : hotel_inform[3] ,\n      "pic_link" : hotel_inform[4] ,\n      "source_rating" : hotel_inform[5] \n    })\n    print(f\' Now in [{idx+1}/{len(hotel_data)}] , finishing rate {(idx+1)/(len(hotel_data))*100} %\')\n    time.sleep(5)\n'

# Get hotels inner-page deatil data ( get directly from the href in main page)

In [ ]:
# get 評語s
from PIL import Image
import requests
from io import BytesIO

restart_number = 1

for idx , hotel_dict in enumerate(hotel_data):
  
  if hotel_dict['room_source'] == 'booking' and idx >= restart_number :

    print(f' now in [{idx+1}/{len(hotel_data)}] ')

    header_url = "https://www.booking.com"
    new_path = header_url + hotel_dict['href']
    res_pic = requests.get(new_path)
    res_pic = res_pic.content
    soup = BeautifulSoup(res_pic)
    print(hotel_dict['store_name'])

    soup_sub = soup.find('div' , { 'class' : " clearfix bh-photo-grid fix-score-hover-opacity "})

    soup_comment = soup_sub.find_all( 'p' , { 'class' : 'trackit althotelsReview2 fixed_review_height fixed_review_top_align review_content'})
    comments = [ item.text.rstrip("\n").strip("\n") for item in soup_comment ] 
    hotel_dict.update({ 'comments' : comments })
    print(comments)

    soup_pic = soup_sub.find_all("a" , { 'class' : 'bh-photo-grid-item'})
    pics = [ item['href'] for item in soup_pic ]
    hotel_dict.update({ 'pics' : pics })
    img = requests.get(pics[0])
    Image.open(BytesIO(img.content))


 now in [3/572] 
Queena Plaza Hotel Tainan
['房間大又舒服', '這是我第二次入住，整體的感覺都好，從入飯店到住宿房間。還有vivispa的按摩。都好喜歡', '房間超大~\n6人房貼心的有兩個化妝枱\n廁所也有兩個馬桶跟洗手台\n餐廳的食物也超美味\n最重要的是超讚的游泳池', '風景美，氣氛佳，設施多，buffet還有大龍蝦，冰淇淋多樣化!!!!!!!!!!!!', '房間非常舒適\n冷氣夠冷\n乾濕分離\n早餐豐盛', '住宿房間乾淨，工作人員細心態度溫和有禮', '整體非常滿意', '早餐多元', '早餐樣多，好吃！\r房間乾淨舒適。', '桂田所有用餐的餐廳我都吃過，都是好吃的，很推薦來這裡用餐。今年的木盒中秋禮盒非常特別，我很喜歡。']
 now in [4/572] 
好事發生 Fortune Inn
['乾淨，民宿主人服務周到', '員工很親切，有問題即時處理\r吹風機不用一直按就能吹，很方便\r沐浴乳跟洗髮精很香，洗完不乾澀\r有停車位且離住宿地很近\r床跟枕頭很舒適，枕頭不會太軟', '棉被輕盈保暖\n桌子很大不怕沒地方放東西\n還有沙發可以休息\n化妝鏡前的椅子舒適好坐\n是皮革的不是木椅', '民宿乾淨又漂亮，很溫馨的感覺，房東人很好！推！\n應有盡有！附近也很安靜！', '接待員很親切，還介紹了當地熱門景點給我們參考\n房間跟照片上一模一樣，很乾淨！另外還附免費停車位很棒：）', '民宿老闆人很好 房間也很舒適', '房間很大', '非常乾淨', '1. 提供免費汽車停車位，盥洗用品提供的很完備\n2. 住宿兩天第一天協助更換用品及整理房間', '房東態度十分親切，回覆速度也很快。環境安靜舒適，設備也都很好，加大床睡得真的很舒服哈哈哈，再去台南一定會再訂☺️☺️☺️']
 now in [5/572] 
W旅棧
['東西很新很不錯', '住宿附近位於主要道路附近很熱鬧，不管是早餐還是晚餐都非常方便。住宿的資訊非常完整，包含停車模式與地點都有詳細影片可以參考，停車場的道路雖然小，但也不至於太難進出，尤其是提供了許多參考方案，降低了不少陌生感，尤其是遇到問題時，馬上回覆跟處理，服務態度超乎預期，是值得推薦的好住宿。', '房間舒適，床舒服好睡', '十分滿意', '佈置氣氛很棒', '環境乾淨，停車方

In [ ]:
def StrToDigit(Number):
 
  Number = ''.join(Number.split(','))
  return int(Number)

dummy = []
for data in hotel_data:
  if data['room_source'] and data['comment_num'] and data['source_rating'] :
    num_in_str = StrToDigit(data["comment_num"].split()[0])
    dummy.append([ float(num_in_str) , float(data['source_rating']) ])

In [ ]:
'''
new_pairs = {}
for pairs in name_transform:
  for ele in hotel_data:
    if pairs[0] == ele['store_name']:
      new_pairs.update({ pairs[0] : pairs[1] })
for dicts in hotel_data:
  if dicts["store_name"] in new_pairs:
    dicts.update({ "source_name" : new_pairs[dicts["store_name"]] })
'''

In [ ]:
# Test for single POST of hotel

In [ ]:
# get hotel name comparision between gmap and booking 
def hotel_name_transform(hotel_data):

  gmap_name = hotel_data['store_name']
  payload_t , headers_t = get_header_payload( ['2020-11-16','2020-11-17'] , target_hotel= gmap_name )
  res_test = requests.post( url, data = payload_t , headers = headers_t)
  res_test = res_test.content
  soup_test = BeautifulSoup( res_test , "html.parser")
  hotel_test = soup_test.find_all('div', {'class':"sr_item_content sr_item_content_slider_wrapper "})

  try:
    booking_name = hotel_test[0].find('span' , { 'class' : "sr-hotel__name "}).string.rstrip("\n").strip("\n")
  except IndexError:
    booking_name = None 
  
  return [gmap_name , booking_name]

def name_modify(name_transform , prev_name , modified_name):

  for ele in name_transform:
    if ele[0] == prev_name:
      ele[1] = modified_name

In [ ]:
#name_transform = []
#for idx , data in enumerate(hotel_data):
#
#  name_transform.append(hotel_name_transform(data))
#  print(f'Now in step [{idx+1}/{len(hotel_data)}]')
#  time.sleep(5)

In [ ]:
#delete action 
#delete_item = {'location': [120.201541, 22.9969523],'rating': 5,'store_name': '趣遊赤崁民宿 Chikan Fun Guesthouse'}
#hotel_data.remove(delete_item)
save_pkl('/content/drive/My Drive/Colab Notebooks/Travel Recommend Project/City data/Tainan/data_hotel' ,hotel_data )

In [ ]:
# modify action 
#name_modify(name_transform,'木本樹台南民宿',"木本樹台南民宿")
save_pkl('/content/drive/My Drive/Colab Notebooks/Travel Recommend Project/City data/Tainan/Hotel_name_transform.pickel' ,name_transform )